<a href="https://colab.research.google.com/github/tam1444AH/Finetuning-Qwen3/blob/main/notebooks/supervised-data-preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install -U "huggingface-hub<1.0" accelerate peft bitsandbytes
!pip -q install -U ms-swift transformers


In [ ]:
import shutil, transformers, huggingface_hub, sys, os
os.environ["PATH"] += ":/root/.local/bin:/usr/local/bin"
print("swift bin:", shutil.which("swift"))
print("ms-swift bin:", shutil.which("ms-swift"))
print("transformers:", transformers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)


In [ ]:
# !pip -q install -U huggingface_hub hf_transfer
!pip -q install -U hf_transfer
!export HF_HUB_ENABLE_HF_TRANSFER=1

import os
from google.colab import userdata
from huggingface_hub import login, whoami
import wandb

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
WANDB_TOKEN = userdata.get('WANDB_KEY')
os.environ["WANDB_API_KEY"] = WANDB_TOKEN
os.environ["HF_TOKEN"] = HF_TOKEN
wandb.login(key=WANDB_TOKEN, relogin=True)
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

In [ ]:
import json, sys

THINK = "<think>\n\n</think>\n\n"

def to_messages(row):
  instruction = row.get("instruction","").strip()
  input = row.get("input","").strip()
  output = row.get("output","").strip()


  user = instruction if not input else f"{instruction}\n\n[Input]\n{input}"

  assistant = THINK + output

  sft_row = {"messages":[{"role":"user","content":user},
                         {"role":"assistant","content":assistant}]}

  return sft_row


In [ ]:
sft_sample_data = open("/content/sft_sample_data.jsonl", "w")
with open("/content/alpaca_instruct_sft_samples2.jsonl") as fin:
  for line in fin:
    if not line.strip(): continue

    row = json.loads(line)
    msg = to_messages(row)

    sft_sample_data.write(json.dumps(msg) + "\n")

sft_sample_data.close()

In [ ]:
%%bash -s "$DATA" "$OUT"
DATA="${1:-sft_sample_data.jsonl}"
OUT="${2:-sft_sample_data_out}"

export SWIFT_USE_HF=1

swift sft \
  --use_hf true \
  --model Qwen/Qwen2.5-Coder-32B \
  --adapters tam2003/peft-FT-2.5-Coder-32b \
  --dataset "$DATA" \
  --output_dir "$OUT" \
  --learning_rate 1e-4 \
  --num_train_epochs 1 \
  --warmup_ratio 0.03 \
  --per_device_train_batch_size 2 \
  --gradient_accumulation_steps 8 \
  --torch_dtype float16 \
  --gradient_checkpointing true \
  --lora_r 16 \
  --lora_alpha 32 \
  --lora_dropout 0.05 \
  --save_steps 200 \
  --logging_steps 20 \
  --loss_scale ignore_empty_think
